# Notebook 1: Ingesting Data into the Feature Store

Specify "Python 3" Kernel, "Data Science" Image.

<div class="alert alert-info"> 💡 <strong> Quick Start </strong>
This notebook will take ~10 minutes to run, so go ahead and click on "Run" in the toolbar, then "Run All Cells". This will run all the cells and begin to ingest data into SageMaker Feature Store which we'll need to call upon in the next few notebooks. As the cells are running, do look through the code and explanations as this will be the foundation for other notebooks.
</div>

### Background

Amazon SageMaker Feature Store is a centralized store for features and associated metadata so features can be easily discovered and reused. You can create an online or an offline store. The online store is used for low latency real-time inference use cases, and the offline store is used for training and batch inference.

The online store is primarily designed for supporting real-time predictions that need low millisecond latency reads and high throughput writes. Offline store is primarily intended for batch predictions and model training. Offline store is an append only store and can be used to store and access historical feature data. The offline store can help you store and serve features for exploration and model training. The online store retains only the latest feature data. Feature Group definitions are immutable after they are created.

There are 5 sets of data about our online grocery use case. Each dataset will have its own Feature Group in SageMaker Feature Store, as described in the image below:

![arch](./img/feature-store-ingestion.png)

We'll be using most Feature Groups to get our data to train our recommendation engine models as well as capture click stream data to influence real-time predictions.

The `click stream historical` Feature Group will be used to initially train our ranking model. As we'll see in later notebooks, the `click stream` Feature Group will be populated by incoming data being streamed via Amazon Kinesis Data Streams (simulating a user interacting with the e-commerce website) and this data will directly influence our recommendation at the time of inference.

### Imports

In [ ]:
import sagemaker
from sagemaker.feature_store.feature_group import FeatureGroup
import boto3
import json
import pandas as pd
from time import gmtime, strftime, time
import time
from parameter_store import ParameterStore
from utils import *
from IPython.core.display import HTML

### Session variables

In [ ]:
role = sagemaker.get_execution_role()
sagemaker_session = sagemaker.Session()
default_bucket = sagemaker_session.default_bucket()
region = sagemaker_session.boto_region_name
s3_client = boto3.client('s3', region_name=region)

# ParameterStore is a custom utility to save local variable values
# for use across all notebooks
ps = ParameterStore(verbose=False)
ps.set_namespace('feature-store-workshop')

In [ ]:
prefix = 'recsys-feature-store'

# Feature Store variables
fs_prefix = 'recsys-'
current_timestamp = strftime('%m-%d-%H-%M', gmtime())
customers_feature_group_name = f'{fs_prefix}customers-fg-{current_timestamp}'
products_feature_group_name = f'{fs_prefix}products-fg-{current_timestamp}'
orders_feature_group_name = f'{fs_prefix}orders-fg-{current_timestamp}'
click_stream_historical_feature_group_name = f'{fs_prefix}click-stream-historical-fg-{current_timestamp}'
click_stream_feature_group_name = f'{fs_prefix}click-stream-fg-{current_timestamp}'

ps.create({'customers_feature_group_name': customers_feature_group_name,
           'products_feature_group_name': products_feature_group_name,
           'orders_feature_group_name': orders_feature_group_name,
           'click_stream_historical_feature_group_name': click_stream_historical_feature_group_name,
           'click_stream_feature_group_name': click_stream_feature_group_name})

In [ ]:
print(f'Using SageMaker version: {sagemaker.__version__}')
print(f'Using SageMaker Role: {role}')
print(f'Using S3 Bucket: {default_bucket}')

In [ ]:
print('Feature groups names:\n')
print(customers_feature_group_name)
print(products_feature_group_name)
print(orders_feature_group_name)
print(click_stream_historical_feature_group_name)
print(click_stream_feature_group_name)

### Load and explore datasets

#### Customers dataset

In [ ]:
df_customers = pd.read_csv('data/customers.csv')
df_customers.head()

#### Products dataset

In [ ]:
df_products = pd.read_csv('data/products.csv')
df_products.head()

#### Orders dataset

In [ ]:
df_orders = pd.read_csv('data/orders.csv')
df_orders.head()

#### Click stream historical dataset

In [ ]:
df_click_stream_historical = pd.read_csv('data/click_stream_historical.csv')
df_click_stream_historical.head()

#### Click stream

In [ ]:
# Read a sample in order to have a schema for Feature Group creation
df_click_stream = pd.read_csv('data/click_stream.csv')
df_click_stream.head()

### Create feature definitions and feature groups

In order to create a Feature Group, you must first create a Feature Definition. A Feature Definition is a schema that describes your data's columns and data types. We'll be inferring this information from the Pandas dataframe itself.

A Feature Group is the main Feature Store resource that contains the metadata for all the data stored in Amazon SageMaker Feature Store. A feature group is a logical grouping of features, defined in the feature store, to describe records. A feature group’s definition is composed of a list of feature definitions, a record identifier name, and configurations for its online and offline store. For our purposes, we'll be using both online and offline stores for our Feature Groups.

For more information, see [Feature Store Concepts](https://docs.aws.amazon.com/sagemaker/latest/dg/feature-store-getting-started.html#feature-store-concepts) and [these docs](https://docs.aws.amazon.com/sagemaker/latest/dg/feature-store-create-feature-group.html).

In [ ]:
customers_feature_group = create_feature_group(df_customers, customers_feature_group_name,
                                               'customer_id', prefix, sagemaker_session)
products_feature_group = create_feature_group(df_products, products_feature_group_name, 'product_id',
                                              prefix, sagemaker_session)
orders_feature_group = create_feature_group(df_orders, orders_feature_group_name, 'order_id', prefix,
                                            sagemaker_session)
click_stream_historical_feature_group = create_feature_group(df_click_stream_historical,
                                                             click_stream_historical_feature_group_name,
                                                             'click_stream_id', prefix, sagemaker_session)
click_stream_feature_group = create_feature_group(df_click_stream, click_stream_feature_group_name, 'customer_id',
                                                  prefix, sagemaker_session)

When feature data is ingested and updated, Feature Store stores historical data for all features in the offline store, whose data is located in S3. You can actually query this offline data using Athena so long as you have the Feature Group's AWS Glue Data Catalog table name. Fortunately, this table name is stored in the metadata for each Feature Group.

Let's go ahead and store our Feature Group table names for later use.

In [ ]:
customers_table = get_feature_group_table_name(customers_feature_group)
products_table = get_feature_group_table_name(products_feature_group)
orders_table = get_feature_group_table_name(orders_feature_group)
click_stream_historical_table = get_feature_group_table_name(click_stream_historical_feature_group)
click_stream_table = get_feature_group_table_name(click_stream_feature_group)

# Store table names locally to be used in other notebooks
ps.add({'customers_table': customers_table,
        'products_table': products_table,
        'orders_table': orders_table,
        'click_stream_historical_table': click_stream_historical_table,
        'click_stream_table': click_stream_table})

### Ingest data into feature groups

Now let's take the data from our Pandas dataframes and ingest them into the Feature Groups we created above.

In [ ]:
ingest_data_into_feature_group(df_customers, customers_feature_group)
customers_count = df_customers.shape[0]

ingest_data_into_feature_group(df_products, products_feature_group)
products_count = df_products.shape[0]

ingest_data_into_feature_group(df_orders, orders_feature_group)
orders_count = df_orders.shape[0]

ingest_data_into_feature_group(df_click_stream_historical, click_stream_historical_feature_group)
click_stream_historical_count = df_click_stream_historical.shape[0]

# Add Feature Group counts for later use
ps.add({'customers_count': customers_count,
        'products_count': products_count,
        'orders_count': orders_count,
        'click_stream_historical_count': click_stream_historical_count,
        'click_stream_count': 0})

This ingestion process is using parallelism but will still take a few minutes for our data to be ingested into each respective Feature Group.

<div class="alert alert-info"> 💡 <strong> Feature Store Ingestion </strong>
In the above code, we were waiting on data to ingest to the online store. When we ingest data into a Feature Group that has the online store enabled, it first gets ingested into the online store so that it can be used in a real-time setting. That ingested data then automatically syncs to the offline store, a process that takes anywhere from 5 to 10 minutes. Below, we're waiting on that data to be written to the offline store so that we can use it in the next notebook.
</div>

In [ ]:
wait_for_offline_data(customers_feature_group_name, df_customers, sagemaker_session)
wait_for_offline_data(products_feature_group_name, df_products, sagemaker_session)
wait_for_offline_data(orders_feature_group_name, df_orders, sagemaker_session)
wait_for_offline_data(click_stream_historical_feature_group_name, df_click_stream_historical, sagemaker_session)

If you click on the link below, you can actually see where this offline data is stored for each Feature Group.

In [ ]:
account_id = default_bucket.split('-')[-1]
offline_store_url = f'https://s3.console.aws.amazon.com/s3/buckets/{default_bucket}?region={region}&prefix={prefix}/{account_id}/sagemaker/{region}/offline-store/&showversions=false'
display(HTML(f"<a href='{offline_store_url}'>Offline Feature Store S3 Link</a>"))

In [ ]:
# Save all our local params
ps.store()

Go back to Workshop Studio and click on "Next".